# DeepseekTravels - Packing List Agent

This notebook demonstrates a LangChain agent integrated with MCP (Model Context Protocol) tools for generating intelligent travel packing lists. The agent uses Azure OpenAI and integrates with weather, travel requirements, and booking services.

## Features
- Weather-aware packing recommendations  
- Trip length and activity-based scaling
- Airline baggage and security restrictions
- Capacity and weight constraint optimization
- Booking suggestions with confirmation flow
- Keep-it-simple mode for quick checklists

**Phase 1**: All MCP servers run in offline/mock mode - no external HTTP calls.

In [ ]:
# Cell 0: Environment Check - Ensure virtual environment is being used
import sys, shutil
print("python:", sys.executable)
print("uv:", shutil.which("uv"))
print("")
print("✅ Environment check complete - should show .venv path for this project")

In [ ]:
# Cell 1: Install Dependencies
# Install uv package manager and all required dependencies
%pip install -U uv
!uv pip install -r packing_requirements.txt
print("")
print("✅ All dependencies installed successfully!")

In [ ]:
# Cell 2: Imports and Setup
import os
from dotenv import load_dotenv
import asyncio
import json
from typing import Dict, Any, List
from datetime import datetime, date

# LangChain imports
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# Official MCP adapter imports for HTTP transport
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Load environment variables for DeepseekTravels
load_dotenv('.env')

# Verify offline mode is enforced
offline_mode = os.getenv('DEEPSEEKTRAVELS_OFFLINE', 'true').lower() == 'true'
use_mocks = os.getenv('DEEPSEEKTRAVELS_USE_MOCKS', 'true').lower() == 'true'

print(f"🛂 DeepseekTravels Configuration:")
print(f"   Offline Mode: {offline_mode}")
print(f"   Use Mocks: {use_mocks}")
print(f"   Phase 1: {'✅ Enforced' if offline_mode and use_mocks else '❌ Live mode detected!'}")
print("")
print("✅ DeepseekTravels imports and configuration loaded successfully!")

In [ ]:
# Cell 3: MCP Client Configuration
# Configure MCP clients for multiple servers (weather, attractions, travel-requirements, booking)

# Global MCP client
mcp_client = None

async def create_mcp_tools():
    """Create MCP tools using the official LangChain MCP adapter with HTTP transport"""
    global mcp_client
    
    try:
        # Define MCP server configurations
        server_configs = {
            "weather": {
                "transport": "streamable_http",
                "url": os.getenv("WEATHER_MCP_URL", "http://127.0.0.1:8009/mcp/")
            },
            "attractions": {
                "transport": "streamable_http",
                "url": os.getenv("ATTRACTIONS_MCP_URL", "http://127.0.0.1:8008/mcp/")
            },
            "travel_requirements": {
                "transport": "streamable_http",
                "url": os.getenv("TRAVEL_REQUIREMENTS_MCP_URL", "http://127.0.0.1:8010/mcp/")
            },
            "booking": {
                "transport": "streamable_http",
                "url": os.getenv("BOOKING_MCP_URL", "http://127.0.0.1:8011/mcp/")
            }
        }
        
        print("🔗 Connecting to MCP servers...")
        for name, config in server_configs.items():
            print(f"   • {name}: {config['url']}")
        
        # Create the multi-server MCP client
        mcp_client = MultiServerMCPClient(server_configs)
        
        # Get available tools from all servers
        tools = await mcp_client.get_tools()
        
        print(f"")
        print(f"🛠️  Loaded {len(tools)} tools from {len(server_configs)} MCP servers")
        
        return tools
        
    except Exception as e:
        print(f"❌ Error connecting to MCP servers: {e}")
        print("   Make sure all MCP servers are running:")
        for name, config in server_configs.items():
            print(f"   • {name}: uv run main.py (in src/mcp/{name.replace('_', '-')}-mcp/)")
        raise

print("✅ MCP client configuration ready!")

In [ ]:
# Cell 4: DeepseekTravels Agent Setup
# Initialize the LangChain agent with Azure OpenAI and MCP tools

# Global agent components
agent_executor = None
tools = []

# DeepseekTravels System Prompt
DEEPSEEKTRAVELS_SYSTEM_PROMPT = """
You are DeepseekTravels, an intelligent travel packing assistant. Your goal is to generate optimized packing lists that consider:

CORE RESPONSIBILITIES:
• Generate comprehensive packing lists based on destination, weather, activities, and trip length
• Respect capacity constraints (backpack volume, weight limits) and airline restrictions
• Integrate weather forecasts to recommend appropriate clothing and gear
• Check security restrictions and baggage rules for specific airlines/routes
• Suggest relevant bookings (flights, hotels, activities) with explicit confirmation required
• Provide keep-it-simple mode for concise, printable checklists

TOOL USAGE ORDER:
1. Get weather forecast for destination and dates
2. Check travel requirements (visas, documents, security restrictions) 
3. Check airline baggage rules if airline specified
4. Generate packing list considering all factors
5. For bookings: search → present summary → get explicit confirmation → book

CONSTRAINTS HANDLING:
• Always ask for capacity (L), max weight (kg), airline, and activity types if not provided
• Scale clothing quantities based on trip length and laundry access
• Remove/replace items to fit within capacity using priority-based selection
• Replace large liquids with travel sizes for airline compliance
• Flag restricted items and suggest alternatives

BOOKING CONFIRMATION GATES:
• NEVER book without explicit user confirmation
• Present structured summary: item, price, dates, cancellation policy
• Ask "Shall I proceed with this booking?" before calling confirm_booking
• Use hold_booking first, then confirm_booking only after user approval

SCOPE GUARDRAILS:
• Focus ONLY on travel planning and packing
• Refuse non-travel requests politely: "I specialize in travel packing assistance"
• No medical/legal advice beyond linking to official sources
• Emphasize mock data warnings in Phase 1

TOKEN EFFICIENCY:
• Batch parallel tool calls where possible (weather + requirements)
• Use structured, terse outputs in keep-it-simple mode
• Summarize long tool results focusing on actionable information
• Limit tool calls per turn (≤4 unless critical)

OUTPUT FORMAT:
• Use clear categories: Clothing, Electronics, Toiletries, Documents, etc.
• Show quantities, estimated weight/volume, and rationale
• Highlight essential vs. nice-to-have items
• Include capacity analysis: "Uses 18L of your 28L backpack (64%)"
• End with key reminders and any restriction warnings

Remember: You are helpful, thorough, and safety-conscious while respecting user constraints and preferences.
"""

async def setup_agent():
    """Setup the DeepseekTravels agent with Azure OpenAI and MCP tools"""
    global agent_executor, tools
    
    try:
        # Load MCP tools
        tools = await create_mcp_tools()
        print(f"🧰 Loaded {len(tools)} MCP tools")
        
        # Initialize Azure OpenAI
        llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_API_VERSION", "2024-12-01-preview"),
            azure_deployment=os.getenv("DEPLOYMENT_NAME", "gpt-5-nano"),
            temperature=1,  # Lower temperature for consistent packing advice
            verbose=True
        )
        
        print(f"🧠 Azure OpenAI initialized: {os.getenv('DEPLOYMENT_NAME', 'gpt-5-nano')}")
        
        # Create prompt template
        prompt = ChatPromptTemplate.from_messages([
            ("system", DEEPSEEKTRAVELS_SYSTEM_PROMPT),
            MessagesPlaceholder("chat_history", optional=True),
            ("human", "{input}"),
            MessagesPlaceholder("agent_scratchpad")
        ])
        
        # Create agent
        agent = create_tool_calling_agent(llm, tools, prompt)
        
        # Create memory
        memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key="output"
        )
        
        # Create agent executor
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            memory=memory,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=10
        )
        
        print(f"🤖 DeepseekTravels agent initialized successfully!")
        return agent_executor
        
    except Exception as e:
        print(f"❌ Error setting up agent: {e}")
        raise

print("✅ Agent setup configuration ready!")

In [ ]:
# Cell 5: Initialize Agent
# Run the async initialization

async def initialize_agent():
    """Initialize the DeepseekTravels agent and display available tools"""
    global agent_executor
    
    print("🚀 Initializing DeepseekTravels Agent...")
    print("")
    
    agent_executor = await setup_agent()
    
    print("")
    print("🛠️  Available Tools:")
    for i, tool in enumerate(tools, 1):
        print(f"   {i:2d}. {tool.name} - {tool.description[:80]}{'...' if len(tool.description) > 80 else ''}")
    
    print("")
    print("✅ DeepseekTravels is ready to help with your packing needs!")

# Run initialization
await initialize_agent()

In [ ]:
# Cell 6: User Input Processing 
# Helper function to process user queries

async def process_user_input(user_input: str):
    """Process user input and return agent response"""
    global agent_executor
    
    if agent_executor is None:
        print("❌ Agent not initialized. Please run the initialization cell first.")
        return
    
    try:
        print(f"🧳 Processing: {user_input}")
        print("=" * 80)
        
        # Execute the agent
        response = await agent_executor.ainvoke({"input": user_input})
        
        print("\n" + "=" * 80)
        print("📋 DeepseekTravels Response:")
        print("=" * 80)
        print(response["output"])
        
        return response["output"]
        
    except Exception as e:
        print(f"❌ Error processing request: {e}")
        return None

print("✅ User input processing ready!")

In [ ]:
# Cell 7: Test MCP Connection
# Test connectivity to all MCP servers and display available tools

async def test_mcp_connection():
    """Test MCP server connections and display tool information"""
    global mcp_client, tools
    
    if mcp_client is None:
        print("❌ MCP client not initialized")
        return
    
    print("🔍 Testing MCP Server Connections...")
    print("")
    
    # Group tools by server/category
    tool_categories = {
        "Weather": [],
        "Travel Requirements": [],
        "Booking": [],
        "Attractions": [],
        "Other": []
    }
    
    for tool in tools:
        name = tool.name.lower()
        if "weather" in name:
            tool_categories["Weather"].append(tool)
        elif any(kw in name for kw in ["baggage", "security", "visa", "document"]):
            tool_categories["Travel Requirements"].append(tool)
        elif any(kw in name for kw in ["flight", "hotel", "booking", "activity"]):
            tool_categories["Booking"].append(tool)
        elif "attraction" in name:
            tool_categories["Attractions"].append(tool)
        else:
            tool_categories["Other"].append(tool)
    
    for category, category_tools in tool_categories.items():
        if category_tools:
            print(f"🔧 {category} Tools ({len(category_tools)}):")
            for tool in category_tools:
                print(f"   • {tool.name}: {tool.description}")
            print("")
    
    print(f"✅ All MCP servers connected - {len(tools)} tools available")

# Test connections
await test_mcp_connection()

In [ ]:
# Cell 8: Example Usage
# Demonstrate the DeepseekTravels agent with example scenarios

print("🌟 DeepseekTravels Example Scenarios")
print("="*50)
print("")

# Example 1: Business trip to Tokyo
print("📊 Example 1: Tokyo Business Trip")
print("-" * 30)
await process_user_input(
    "I'm traveling to Tokyo for a 5-day business trip in November. "
    "I have a 28L backpack, flying ANA economy class, with business meetings and some evening events. "
    "What should I pack?"
)

print("\n" + "="*80 + "\n")

# Example 2: Weekend beach trip  
print("🏖️ Example 2: Lisbon Beach Weekend")
print("-" * 30)
await process_user_input(
    "Weekend trip to Lisbon in late September, mostly beach and casual sightseeing. "
    "Small 20L daypack, can do laundry. Keep it simple!"
)

In [ ]:
# Cell 9: Interactive Chat Loop
# Interactive conversation with DeepseekTravels

async def chat_with_deepseektravels():
    """Interactive chat loop with the DeepseekTravels agent"""
    print("🗣️  Starting interactive chat with DeepseekTravels")
    print("   Type 'exit', 'quit', or 'bye' to end the conversation")
    print("   Type 'help' for example questions")
    print("=" * 80)
    
    while True:
        try:
            # Get user input
            user_input = input("\n🧳 You: ").strip()
            
            # Check for exit conditions
            if user_input.lower() in ['exit', 'quit', 'bye', 'stop']:
                print("\n👋 DeepseekTravels: Safe travels! Feel free to come back anytime for packing advice.")
                break
            
            # Help command
            if user_input.lower() == 'help':
                print("\n💡 Example questions you can ask:")
                print("   • What should I pack for a week in Iceland in winter?")
                print("   • I need a packing list for Patagonia hiking, 15kg limit")
                print("   • Business trip to Singapore, what are the visa requirements?")
                print("   • Search flights from NYC to Tokyo for November")
                print("   • Keep it simple: beach weekend in Mexico")
                continue
            
            # Skip empty inputs
            if not user_input:
                continue
            
            # Process the input
            print(f"\n🤖 DeepseekTravels: ", end="")
            await process_user_input(user_input)
            
        except KeyboardInterrupt:
            print("\n\n👋 DeepseekTravels: Chat interrupted. Safe travels!")
            break
        except Exception as e:
            print(f"\n❌ Error in chat: {e}")
            continue

print("✅ Interactive chat ready! Run the cell below to start chatting.")
print("\n📝 Or try these quick commands here:")
print("   await process_user_input('Your question here')")

In [ ]:
# Cell 10: Start Interactive Chat
# Uncomment the line below to start interactive chat

await chat_with_deepseektravels()

# Or test individual queries:
# await process_user_input("I'm going to Patagonia for 10 days of hiking. What gear should I bring? My backpack is 65L and I can carry up to 20kg.")

In [ ]:
# Cell 11: Cleanup
# Clean up MCP connections and resources

async def cleanup_mcp():
    """Clean up MCP client connections"""
    global mcp_client, agent_executor
    
    try:
        if mcp_client:
            print("🧹 Cleaning up MCP connections...")
            await mcp_client.close()
            mcp_client = None
            print("   ✅ MCP client closed")
        
        agent_executor = None
        print("   ✅ Agent executor cleared")
        
        print("\n🎯 DeepseekTravels session ended. Thank you for using our packing assistant!")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {e}")

# Uncomment to run cleanup
# await cleanup_mcp()

print("✅ Cleanup function ready - uncomment the line above to run cleanup when finished")